In [37]:
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, Wav2Vec2ForCTC, Wav2Vec2Processor, TextStreamer
import torch

import json
import os
import re
from typing import Dict, List
import logging
import pickle
from datetime import datetime

tqdm.pandas()

In [2]:
rools = '\n'.join(map(str, json.load(open('data/resume_rools.json'))))

In [185]:
rools = '\n'.join(map(str, rools))

In [35]:
MAX_CHUNKS_LEN = 2048
OVERLAP_LEN = 128

In [3]:
INFERENCE_MODEL_NAME = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(INFERENCE_MODEL_NAME)

In [4]:
inference_model = AutoModelForCausalLM.from_pretrained(
    INFERENCE_MODEL_NAME, 
    torch_dtype=torch.bfloat16,
    device_map="balanced",
    max_memory={0: '10GB', 1: '10GB', 2: '10GB'}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
prompt = """
Ты — ИИ-модератор резюме. Проверь текст резюме на соответствие следующим правилам. Если есть нарушения, верни JSON-объект с типом нарушения и фрагментом текста, который нарушает правило. Если нарушений нет, верни статус "OK".

**Правила:** 
{rools}

**Инструкции:**

- Тщательно проанализируй каждое предложение в резюме.
- Если нарушений нет, верни "status": "OK" и пустой массив violated_rules.
- Если фрагмент нарушает правило, укажи его точную цитату в resume_fragment.
- Ответ должен содержать твои рассуждения по каждому правилу оканчивающийся вердиктом, затем должен быть результат в формате JSON, без Markdown и комментариев.

Формат ответа: **Рассуждения:**, **Результат:** { "status": "OK" | "violation", "violated_rules": [] | [ { "id": "rule_id", "condition": "Текст условия правила на русском", "resume_fragment": "Точная цитата из резюме" } ] }

**Текст резюме:**
{resume_text}
"""

In [38]:
def split_text_with_overlap(text: str, tokenizer: AutoTokenizer, max_length: int, overlap: int) -> List[str]:
    tokens = tokenizer.encode(text, add_special_tokens=False)
    
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + max_length
        chunks.append(tokens[start:end])
        start = end - overlap
        
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

In [6]:
def parse_answer(response_str: str) -> dict:
    parts = response_str.split("**Результат:**")
    
    reasoning = parts[0].replace("**Рассуждения:**\n", "").strip()

    json_match = re.search(r'```json\n(.*?)\n```', parts[1], re.DOTALL)
    json_str = json_match.group(1) if json_match else '{}'

    try:
        result_dict = json.loads(json_str)
    except json.JSONDecodeError:
        result_dict = {"error": "Invalid JSON format"}
    
    return {
        "reasoning": reasoning,
        "result": result_dict
    }

In [7]:
streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True,
    skip_special_tokens=True
)

In [121]:
def get_answer(resume_text: str) -> Dict[str, str]:
    
    text_chunks = split_text_with_overlap(
        resume_text, 
        tokenizer,
        max_length=2048,
        overlap=128
    )
    
    all_reasoning = []
    all_violations = []
    overall_status = "OK"
    
    for chunk in text_chunks:
        formatted_prompt = prompt.replace('{rools}', rools).replace('{resume_text}', chunk)
        
        messages = [
            {"role": "system", "content": "Твоя задача - анализировать части резюме."},
            {"role": "user", "content": formatted_prompt}
        ]
        
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        model_inputs = tokenizer([text], return_tensors="pt").to(inference_model.device)
        
        generated_ids = inference_model.generate(
            **model_inputs,
            max_new_tokens=2048,
            do_sample=False,
            top_p=None,
            top_k=None,
            temperature=None
        )
        
        generated_ids = [
            output_ids[len(input_ids):] 
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        answer_content = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        parsed = parse_answer(answer_content)
        
        all_reasoning.append(parsed['reasoning'])
        if parsed['result']['status'] == 'violation':
            overall_status = 'violation'
            all_violations.extend(parsed['result']['violated_rules'])
    
    return {
        'reasoning': "\n\n".join(all_reasoning),
        'result': {
            'status': overall_status,
            'violated_rules': all_violations
        }
    }

In [52]:
df = pd.read_csv('./data/processed_resume.csv')

In [55]:
test_llm_resp = get_answer(df.text_data.iloc[0])

In [56]:
test_llm_resp

{'reasoning': '1. **rule_1**: Резюме содержит информацию о трудовой деятельности, что соответствует правилу, так как указана должность и период работы.\n2. **rule_2**: В резюме указана должность "водитель", которая является реальной профессией, поэтому правило соблюдается.\n3. **rule_4**: Резюме написано на русском языке, что соответствует правилу.\n4. **rule_5**: Информация о работодателе содержит только имя и фамилию, что не является нарушением, так как это стандартная практика для ИП.\n5. **rule_6**: Все информация в резюме относится к соискателю, что соответствует правилу.\n6. **rule_7**: В резюме отсутствуют маты и токсичные выражения, что соответствует правилу.',
 'result': {'status': 'OK', 'violated_rules': []}}

In [57]:
df['t_pro_answer'] = df['text_data'].progress_apply(get_answer)

  2%|▏         | 15/905 [07:55<7:49:44, 31.67s/it]

KeyboardInterrupt



In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('processing.log'),
        logging.StreamHandler()
    ]
)

def process_dataframe(df, checkpoint_interval=20):
    last_checkpoint = get_last_checkpoint()
    start_index = 0
    results = []
    
    if last_checkpoint:
        start_index = last_checkpoint['index'] + 1
        results = last_checkpoint['results']
        logging.info(f"Resuming from index {start_index}")

    try:
        for i in tqdm(range(start_index, len(df)), initial=start_index, total=len(df)):
            try:
                result = get_answer(df.text_data[i])
                results.append(result)
                
                if (i - start_index) % checkpoint_interval == 0:
                    save_checkpoint(i, results)
                    
            except Exception as e:
                logging.error(f"Error processing row {i}: {str(e)}")
                results.append({'error': str(e)})
                
        save_checkpoint(len(df)-1, results)
        return pd.DataFrame(results)
        
    except KeyboardInterrupt:
        logging.info("Process interrupted. Saving last checkpoint...")
        save_checkpoint(i, results)
        return pd.DataFrame(results)

def save_checkpoint(index, results):
    checkpoint = {
        'timestamp': datetime.now(),
        'index': index,
        'results': results
    }
    
    with open('checkpoint.pkl', 'wb') as f:
        pickle.dump(checkpoint, f)
        
    logging.info(f"Checkpoint saved at index {index}")

def get_last_checkpoint():
    if os.path.exists('checkpoint.pkl'):
        try:
            with open('checkpoint.pkl', 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            logging.error(f"Error loading checkpoint: {str(e)}")


result_df = process_dataframe(df)
result_df.to_csv('final_results.csv', index=False)

2025-02-22 23:18:12,735 - INFO - Resuming from index 10
2025-02-22 23:18:35,163 - INFO - Checkpoint saved at index 10
  1%|▏         | 12/905 [00:57<7:26:36, 30.01s/it]

In [158]:
df['t_pro_answer'] = result_df['result']

In [159]:
df['t_pro_status'] = result_df['result'].apply(lambda el: el['status'])

In [160]:
df['t_pro_reasoning'] = result_df['reasoning']

In [198]:
df['t_pro_status'].value_counts()

t_pro_status
OK           828
violation     77
Name: count, dtype: int64

In [199]:
df.loc[df['Результат'] == 'Отклонено', 't_pro_status']

198    violation
242    violation
425    violation
428    violation
822    violation
Name: t_pro_status, dtype: object

In [201]:
df.loc[df['Результат'] == 'Принято', 't_pro_status'].value_counts()

t_pro_status
OK           690
violation     59
Name: count, dtype: int64

In [206]:
false_negative = (df.loc[df['Результат'] == 'Отклонено', 't_pro_status'] == 'OK').astype(int).sum()
false_positive = (df.loc[df['Результат'] == 'Принято', 't_pro_status'] == 'violation').astype(int).sum()
recall = (df.loc[df['Результат'] == 'Отклонено', 't_pro_status'] == 'violation').astype(int).mean()
precision = (df.loc[df['Результат'] == 'Принято', 't_pro_status'] == 'OK').astype(int).mean()
f1_score = 2 * (recall * precision) / (recall + precision)

In [208]:
print(f'false negative: {false_negative}')
print(f'false positive: {false_negative}')
print(f'recall: {recall:.2f}')
print(f'precision: {precision:.2f}')
print(f'f1-score: {f1_score:.2f}')

false negative: 0
false positive: 0
recall: 1.00
precision: 0.92
f1-score: 0.96


In [225]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .ipynb_checkpoints/3_extract_rools-checkpoint.ipynb
	new file:   .ipynb_checkpoints/5_local_inference-checkpoint.ipynb
	modified:   3_extract_rools.ipynb
	modified:   5_local_inference.ipynb
	new file:   checkpoint.pkl
	modified:   data/resume_rools.json
	new file:   final_results.csv
	new file:   processing.log



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
